In [1]:
import pandas as pd
import numpy as np
import os

# Scraping Wikipedia Page to Dataframe

In [2]:
#read wikipedia into pandas dataframe
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
#extract target table
df=tables[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
#rename columns
df.columns=['PostalCode', 'Borough', 'Neighborhood']

In [5]:
#drop "Not assigned" in Borough column
postalcode=df.replace('Not assigned', np.NaN).dropna(subset=['Borough'], axis=0).reset_index(drop=True)
postalcode

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
#There is no "Not assigned" neighborhood remaining
postalcode.loc[postalcode['Neighborhood'] == np.NaN]

,PostalCode,Borough,Neighborhood


In [7]:
# There is no duplicate in PostalCode
postalcode['PostalCode'].unique

<bound method Series.unique of 0      M3A
1      M4A
2      M5A
3      M6A
4      M7A
      ... 
98     M8X
99     M4Y
100    M7Y
101    M8Y
102    M8Z
Name: PostalCode, Length: 103, dtype: object>

In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(postalcode['Borough'].unique()),
        postalcode.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [9]:
postalcode.shape

(103, 3)

# Getting Latitude and Longitude Coordinates of Each Neighborhood

In [10]:
# Install geocoder
#conda install -c conda-forge geocoder


In [11]:
#Create function get_ll to obtain Latitude and Longitude Coordinates with postalcode
import geocoder # import geocoder

def get_ll (postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return [latitude, longitude]

In [12]:
# Adding two columns for coordinates
postalcode['Latitude'] = np.NaN
postalcode['Longitude'] = np.NaN

In [13]:
# Adding Latitude and Longitude Coordinates to each row of postalcode dataframe

for index in postalcode.index:
    ll=get_ll(postalcode.at[index, 'PostalCode'])
    postalcode.at[index, 'Latitude']=ll[0]
    postalcode.at[index, 'Longitude']=ll[1]


In [14]:
postalcode

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


In [15]:
# Save to csv file so that we don't need to retrive coordinates everytime 
postalcode.to_csv('postalcode')

# Explore Neighborhoods in Toronto

In [16]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Install geopy
#!conda install -c conda-forge geopy --yes
# Install folium
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import json
import requests # library to handle requests

In [17]:
# Use geopy library to get the latitude and longitude values of Toronto.

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
toronto_data=pd.read_csv('postalcode', index_col=0)
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


In [19]:
# create map of New York using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
# Eneter API credentials and version
CLIENT_ID =  os.environ.get('FOURSQUARE_CLIENT_ID') # your Foursquare ID
CLIENT_SECRET = os.environ.get('FOURSQUARE_CLIENT_SECRET') # your Foursquare Secret
ACCESS_TOKEN = os.environ.get('FOURSQUARE_ACCESS_TOKEN') # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
'''print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)'''

"print('Your credentails:')\nprint('CLIENT_ID: ' + CLIENT_ID)\nprint('CLIENT_SECRET:' + CLIENT_SECRET)"

In [21]:
# create a function to repeat the same process to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(1352, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store


In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,13,13,13,13,13,13
"Alderwood, Long Branch",4,4,4,4,4,4
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
...,...,...,...,...,...,...
"Willowdale, Willowdale West",4,4,4,4,4,4
Woburn,4,4,4,4,4,4
Woodbine Heights,18,18,18,18,18,18


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


# Analyze Each Neighborhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# get a list of columns
cols = list(toronto_onehot)
# move the Neighborhood column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighborhood')))

In [28]:
# move neighborhood column to the first column
toronto_onehot = toronto_onehot[cols]

toronto_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [30]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Discount Store,Department Store,Skating Rink,Supermarket,Sushi Restaurant,Bubble Tea Shop,Bakery,Badminton Court,Shopping Mall
1,"Alderwood, Long Branch",Performing Arts Venue,Gym,Convenience Store,Pub,Discount Store,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Business Service,Yoga Studio,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,Bayview Village,Trail,Construction & Landscaping,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Cosmetics Shop,Pub,Butcher,Juice Bar,Restaurant,Café,Sports Club
...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",Park,Grocery Store,Pizza Place,Coffee Shop,Discount Store,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
94,Woburn,Park,Korean BBQ Restaurant,Coffee Shop,Business Service,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
95,Woodbine Heights,Grocery Store,Pharmacy,Café,Bus Line,Middle Eastern Restaurant,Dance Studio,Optical Shop,Breakfast Spot,Metro Station,Coffee Shop
96,York Mills West,Park,Coffee Shop,Speakeasy,Convenience Store,Cosmetics Shop,Donut Shop,Field,Fast Food Restaurant,Farmers Market,Farm


# Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 5 clusters.

In [32]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [33]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 4, 3, 3, 3, 3, 3, 3])

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data



In [35]:
# merge manhattan_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,2.0,Park,Food & Drink Shop,Pool,Yoga Studio,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.73057,-79.31306,2.0,Pharmacy,Grocery Store,Park,Yoga Studio,Discount Store,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,3.0,Coffee Shop,Breakfast Spot,Yoga Studio,Spa,Bakery,Distribution Center,Electronics Store,Event Space,Food Truck,Gym / Fitness Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,3.0,Clothing Store,Cosmetics Shop,Women's Store,Food Court,American Restaurant,Coffee Shop,Electronics Store,Shopping Mall,Leather Goods Store,Mediterranean Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,3.0,Coffee Shop,Sandwich Place,Italian Restaurant,Bank,Burrito Place,Portuguese Restaurant,Mediterranean Restaurant,Theater,Café,Fried Chicken Joint
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113,0.0,Lounge,Park,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133,3.0,Coffee Shop,Gay Bar,Men's Store,Japanese Restaurant,Steakhouse,Dog Run,Ice Cream Shop,Bookstore,Pub,Breakfast Spot
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544,3.0,Café,Coffee Shop,Theater,Restaurant,Steakhouse,Hotel,Concert Hall,Japanese Restaurant,Asian Restaurant,Speakeasy
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945,3.0,Bank,Coffee Shop,Park,Flower Shop,Sushi Restaurant,Italian Restaurant,Chinese Restaurant,Fast Food Restaurant,Deli / Bodega,Department Store


It seems Foursquare does not have data for 3 neighborhoods (see last three rows below with NaN in "Cluster Labels"). Therefore they need to be dropped

In [36]:
toronto_merged.sort_values(['Cluster Labels'])

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M2H,North York,Hillcrest Village,43.80225,-79.35558,0.0,Park,Residential Building (Apartment / Condo),Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113,0.0,Lounge,Park,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
45,M2L,North York,"York Mills, Silver Hills",43.75698,-79.38060,0.0,Park,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
68,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.69479,-79.41440,0.0,Park,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
69,M6P,West Toronto,"High Park, The Junction South",43.65973,-79.46281,0.0,Park,Sandwich Place,Residential Building (Apartment / Condo),Convenience Store,Cosmetics Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,M4H,East York,Thorncliffe Park,43.70142,-79.34905,3.0,Yoga Studio,Housing Development,Park,Pizza Place,Coffee Shop,Sandwich Place,Burger Joint,Supermarket,Indian Restaurant,Bridge
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,3.0,Gym,Thai Restaurant,Gym / Fitness Center,Pet Store,Miscellaneous Shop,Eastern European Restaurant,Mattress Store,Burrito Place,Burger Joint,Sushi Restaurant
39,M2K,North York,Bayview Village,43.78112,-79.38060,4.0,Trail,Construction & Landscaping,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
26,M1H,Scarborough,Cedarbrae,43.76944,-79.23892,4.0,Trail,Yoga Studio,Distribution Center,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [37]:
toronto_merged=toronto_merged.dropna(subset=['Cluster Labels'])
toronto_merged=toronto_merged.astype({'Cluster Labels':'int32'})

In [38]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [39]:
# visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

## Cluster 1

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,North York,0,Park,Residential Building (Apartment / Condo),Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
45,North York,0,Park,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
68,Central Toronto,0,Park,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
69,West Toronto,0,Park,Sandwich Place,Residential Building (Apartment / Condo),Convenience Store,Cosmetics Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
98,Etobicoke,0,Lounge,Park,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


## Cluster 2

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,1,Fast Food Restaurant,Yoga Studio,Fish Market,Field,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
56,York,1,Fast Food Restaurant,Playground,Coffee Shop,Yoga Studio,Dog Run,Field,Farmers Market,Farm,Falafel Restaurant,Event Space


## Cluster 3

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Food & Drink Shop,Pool,Yoga Studio,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,North York,2,Pharmacy,Grocery Store,Park,Yoga Studio,Discount Store,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,Etobicoke,2,Pharmacy,Park,Shopping Mall,Café,Bank,Skating Rink,Grocery Store,Cosmetics Shop,Dog Run,Farm
16,York,2,Park,Hockey Arena,Field,Playground,Trail,Grocery Store,Falafel Restaurant,Farm,Farmers Market,Discount Store
17,Etobicoke,2,Fish & Chips Shop,Shopping Mall,College Rec Center,Carpet Store,Electronics Store,Park,Grocery Store,Gluten-free Restaurant,Farm,Falafel Restaurant
18,Scarborough,2,Construction & Landscaping,Gym / Fitness Center,Park,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
22,Scarborough,2,Park,Korean BBQ Restaurant,Coffee Shop,Business Service,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
32,Scarborough,2,Park,Grocery Store,Restaurant,Indian Restaurant,Cosmetics Shop,Creperie,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
35,East York,2,Intersection,Park,Home Service,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
38,Scarborough,2,Park,Playground,Coffee Shop,Department Store,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


## Cluster 4

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Breakfast Spot,Yoga Studio,Spa,Bakery,Distribution Center,Electronics Store,Event Space,Food Truck,Gym / Fitness Center
3,North York,3,Clothing Store,Cosmetics Shop,Women's Store,Food Court,American Restaurant,Coffee Shop,Electronics Store,Shopping Mall,Leather Goods Store,Mediterranean Restaurant
4,Downtown Toronto,3,Coffee Shop,Sandwich Place,Italian Restaurant,Bank,Burrito Place,Portuguese Restaurant,Mediterranean Restaurant,Theater,Café,Fried Chicken Joint
7,North York,3,Coffee Shop,Park,Gas Station,Bubble Tea Shop,Supermarket,Beer Store,Intersection,Clothing Store,Burger Joint,Grocery Store
8,East York,3,Pizza Place,Fast Food Restaurant,Intersection,Rock Climbing Spot,Bank,Breakfast Spot,Café,Flea Market,Gastropub,Athletics & Sports
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,3,Café,Restaurant,Japanese Restaurant,Coffee Shop,Seafood Restaurant,American Restaurant,Beer Bar,Gym / Fitness Center,Gluten-free Restaurant,Gastropub
99,Downtown Toronto,3,Coffee Shop,Gay Bar,Men's Store,Japanese Restaurant,Steakhouse,Dog Run,Ice Cream Shop,Bookstore,Pub,Breakfast Spot
100,East Toronto,3,Café,Coffee Shop,Theater,Restaurant,Steakhouse,Hotel,Concert Hall,Japanese Restaurant,Asian Restaurant,Speakeasy
101,Etobicoke,3,Bank,Coffee Shop,Park,Flower Shop,Sushi Restaurant,Italian Restaurant,Chinese Restaurant,Fast Food Restaurant,Deli / Bodega,Department Store


## Cluster 5

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Scarborough,4,Trail,Yoga Studio,Distribution Center,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
39,North York,4,Trail,Construction & Landscaping,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
